# Citrus Disease Classification

> ### Importing Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
import cv2
from sklearn.model_selection import train_test_split
import pandas as pd
import pathlib


> ### Checking GPUs Available

In [ ]:

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

> ### Creating training and testing dataframes for training

In [ ]:
# Define the path to the dataset
data_dir = '/kaggle/input/orange-leaf-disease-dataset/train/train'
data_dir = pathlib.Path(data_dir)

# Create a dictionary of class labels and corresponding image paths
objects = {
    'Citrus_Canker': list(data_dir.glob('Citrus_Canker_Diseases_Leaf_Orange/*')),
    'Nutrient_Deficiency': list(data_dir.glob('Citrus_Nutrient_Deficiency_Yellow_Leaf_Orange/*')),
    'Healthy_Leaf_Orange': list(data_dir.glob('Healthy_Leaf_Orange/*')),
    'Multiple_Diseases': list(data_dir.glob('Multiple_Diseases_Leaf_Orange/*')),
    'Young_Healthy': list(data_dir.glob('Young_Healthy_Leaf_Orange/*'))
}

# Define the class labels
objects_labels = {
    'Citrus_Canker': 0,
    'Nutrient_Deficiency': 1,
    'Healthy_Leaf_Orange': 2,
    'Multiple_Diseases': 3,
    'Young_Healthy': 4
}

# Create lists to store the image paths and labels
image_paths, y = [], []

# Collect the image paths and labels
for name, images in objects.items():
    for image in images:
        image_paths.append(str(image))
        y.append(objects_labels[name])

# Split the data into training and testing sets
image_paths_train, image_paths_test, y_train, y_test = train_test_split(image_paths, y, test_size=0.1, random_state=0)





# Create a DataFrame to store image paths and labels
train_df = pd.DataFrame({'image_path': image_paths_train, 'label': y_train})
test_df = pd.DataFrame({'image_path': image_paths_test, 'label': y_test})


> ### Data Augmentation

In [ ]:
# Create data generators for training and testing
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Scale to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Scale to [0, 1]

# Set batch size and number of classes
batch_size = 32
num_classes = 5

# Create data iterators for training and testing
train_iterator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='label',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='raw',
    shuffle=True
)

test_iterator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='label',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='raw',
    shuffle=False
)


> ### Building the deep learning model

In [ ]:
# Build the CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    # Add more layers as needed
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Training the model
epochs = 100
model.fit(
    train_iterator,
    steps_per_epoch=len(train_df) // batch_size,
    epochs=epochs
)

# Evaluate the model on the test set
model.evaluate(
    test_iterator,
    steps=len(test_df) // batch_size
)



> ### Saving the model and exporting it to an H5 File

In [ ]:
# Save the model
model.save('lung_cancer_classification_model.h5')